BT4014 MMA Project

In [1]:
import pandas as pd
import datetime

In [2]:
# Using the basic stats as outlined in the dataset description (except urls)
masterMLpublic = pd.read_csv("/kaggle/input/mma-differentials-and-elo/masterMLpublic.csv",
                            usecols = [
                                "date", "result", "fighter", "opponent", "division",
                                "stance", "dob", "method", "total_comp_time", "round", 
                                "time", "referee", "time_format", "reach", "height",
                                "age", "knockdowns", "sub_attempts", "reversals", "control",
                                "takedowns_landed", "takedowns_attempts",
                                "sig_strikes_landed", "sig_strikes_attempts",
                                "total_strikes_landed", "total_strikes_attempts",
                                "head_strikes_landed", "head_strikes_attempts",
                                "body_strikes_landed", "body_strikes_attempts",
                                "leg_strikes_landed", "leg_strikes_attempts",
                                "distance_strikes_landed", "distance_strikes_attempts",
                                "clinch_strikes_landed", "clinch_strikes_attempts",
                                "ground_strikes_landed", "ground_strikes_attempts",
                                "KO_losses", "days_since_last_comp", "lose_streak",
                                "win_streak", "win_loss_ratio", "total_comp_time", "stamina",
                                "num_fights", "trueskill", "elo"
                            ])
# Convert the date columns from object into datetime
masterMLpublic["date"] = pd.to_datetime(masterMLpublic["date"]).dt.date
masterMLpublic["dob"] = pd.to_datetime(masterMLpublic["dob"]).dt.date
masterMLpublic

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
0,1994-03-11,0,sean daugherty,scott morris,NaN,1975-12-04,Open Weight,SUB,1,0:20,...,0,1.0,0,1,0.000000,0,20.0,0.000000,1147.756618,20.794729
1,1994-03-11,1,scott morris,sean daugherty,Orthodox,NaT,Open Weight,SUB,1,0:20,...,0,1.0,1,0,1.000000,0,20.0,0.000000,1252.243382,29.205271
2,1994-03-11,0,ray wizard,patrick smith,NaN,NaT,Open Weight,SUB,1,0:58,...,0,1.0,0,1,0.000000,0,58.0,0.000000,1147.164966,20.794729
3,1994-03-11,1,patrick smith,ray wizard,Orthodox,1963-08-28,Open Weight,SUB,1,0:58,...,0,1.0,1,0,1.000000,0,58.0,0.000000,1249.442138,29.205271
4,1994-03-11,0,david levicki,johnny rhodes,NaN,NaT,Open Weight,KO/TKO,1,12:13,...,1,1.0,0,1,0.000000,1,733.0,0.000000,1141.838344,20.794729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14125,2023-04-15,1,azamat murzakanov,dustin jacoby,Southpaw,1989-04-12,Light Heavyweight,U-DEC,3,5:00,...,0,3.0,3,0,1.000000,0,2222.0,1.000000,1341.922237,35.760448
14126,2023-04-15,0,billy quarantillo,edson barboza,Orthodox,1988-12-08,Featherweight,KO/TKO,1,2:37,...,0,8.0,0,1,0.625000,1,5352.0,0.600000,1264.744398,31.425332
14127,2023-04-15,1,edson barboza,billy quarantillo,Orthodox,1986-01-21,Featherweight,KO/TKO,1,2:37,...,1,28.0,1,0,0.607143,4,18947.0,0.631579,1350.043973,34.795952
14128,2023-04-15,0,arnold allen,max holloway,Southpaw,1994-01-22,Featherweight,U-DEC,5,5:00,...,0,11.0,0,1,0.909091,0,8823.0,0.888889,1489.695647,39.967508


In [3]:
masterMLpublic.dtypes

# Need to figure out what to do with the objects. Contextual bandit -> linear function (i.e., needs some number)
# 1. Label encode?
# 2. Ignore?

# Objects are:
# date        -> ignore
# fighter     -> ignore
# opponent    -> ignore
# stance      -> label encode
# dob         -> ignore (use age instead)
# division    -> ignore (since we only compare within a division)
# method      -> label encode
# time        -> convert to seconds
# referee     -> ignore
# time_format -> label encode

date                          object
result                         int64
fighter                       object
opponent                      object
stance                        object
dob                           object
division                      object
method                        object
round                          int64
time                          object
referee                       object
time_format                   object
reach                        float64
age                          float64
height                       float64
knockdowns                     int64
days_since_last_comp         float64
sub_attempts                   int64
reversals                      int64
control                        int64
takedowns_landed               int64
takedowns_attempts             int64
sig_strikes_landed             int64
sig_strikes_attempts           int64
total_strikes_landed           int64
total_strikes_attempts         int64
head_strikes_landed            int64
h

In [4]:
masterMLpublic.isna().sum()             # count of na
# masterMLpublic.isna().mean() * 100    # percentage of na

# Need to figure out what to do with null values.

# If {variable} is null -> {action}:
# stance  -> remove
# dob     -> remove
# referee -> ignore
# reach   -> impute with division average
# age     -> remove
# height  -> remove

date                            0
result                          0
fighter                         0
opponent                        0
stance                         97
dob                           235
division                        0
method                          0
round                           0
time                            0
referee                        64
time_format                     0
reach                        1282
age                           235
height                         14
knockdowns                      0
days_since_last_comp            0
sub_attempts                    0
reversals                       0
control                         0
takedowns_landed                0
takedowns_attempts              0
sig_strikes_landed              0
sig_strikes_attempts            0
total_strikes_landed            0
total_strikes_attempts          0
head_strikes_landed             0
head_strikes_attempts           0
body_strikes_landed             0
body_strikes_a

In [5]:
masterMLpublic["days_since_last_comp"].value_counts()
# Need to impute value for erroneous days_since_last_comp value (likely caused when fighters have no prior fights)
# Can replace erroneous values with age

days_since_last_comp
216.108044     2357
133.000000      403
140.000000      376
126.000000      376
147.000000      352
               ... 
282.000000        1
2422.000000       1
247.000000        1
1806.000000       1
1004.000000       1
Name: count, Length: 701, dtype: int64

In [6]:
# for col in masterMLpublic.columns:
#     print(col)

In [7]:
# masterdataframe = pd.read_csv("/kaggle/input/mma-differentials-and-elo/masterdataframe.csv")
# masterdataframe

In [8]:
# for col in masterdataframe.columns:
#     print(col)

In [9]:
masterMLpublic["division"].value_counts()
# Can focus on Lightweight division first (since fighters only fight within their division)

division
Lightweight              2470
Welterweight             2410
Middleweight             1898
Featherweight            1340
Heavyweight              1318
Light Heavyweight        1284
Bantamweight             1198
Flyweight                 582
Women's Strawweight       516
Women's Flyweight         380
Women's Bantamweight      362
Open Weight               204
Catch Weight              114
Women's Featherweight      52
Super Heavyweight           2
Name: count, dtype: int64

In [10]:
lightweight = masterMLpublic[masterMLpublic["division"] == "Lightweight"]
lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
198,1997-02-07,0,rainy martinez,jerry bohlander,Orthodox,NaT,Lightweight,SUB,1,1:24,...,0,1.0,0,1,0.000000,0,84.0,0.000000,1155.936792,21.246039
199,1997-02-07,1,jerry bohlander,rainy martinez,Orthodox,1974-02-12,Lightweight,SUB,1,1:24,...,0,4.0,2,0,0.750000,1,1858.0,0.000000,1292.341803,29.965180
200,1997-02-07,0,wallid ismail,yoshiki takahashi,Orthodox,1968-02-23,Lightweight,U-DEC,2,3:00,...,0,1.0,0,1,0.000000,0,900.0,0.000000,1147.756618,20.794729
201,1997-02-07,1,yoshiki takahashi,wallid ismail,Southpaw,1969-03-13,Lightweight,U-DEC,2,3:00,...,10,1.0,1,0,1.000000,0,900.0,0.000000,1252.243382,29.205271
206,1997-02-07,0,nick sanzo,jerry bohlander,NaN,NaT,Lightweight,SUB,1,0:39,...,0,1.0,0,1,0.000000,0,39.0,0.000000,1162.881928,21.710767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2023-03-18,1,justin gaethje,rafael fiziev,Orthodox,1988-11-14,Lightweight,M-DEC,3,5:00,...,0,11.0,1,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578
14108,2023-04-15,0,lando vannata,daniel zellhuber,Orthodox,1992-03-14,Lightweight,U-DEC,3,5:00,...,2,13.0,0,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970
14109,2023-04-15,1,daniel zellhuber,lando vannata,Switch,1999-07-07,Lightweight,U-DEC,3,5:00,...,51,2.0,1,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804
14118,2023-04-15,0,clay guida,rafa garcia,Orthodox,1981-12-08,Lightweight,U-DEC,3,5:00,...,0,35.0,0,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986


In [11]:
lightweight.iloc[0]

date                              1997-02-07
result                                     0
fighter                       rainy martinez
opponent                     jerry bohlander
stance                              Orthodox
dob                                      NaT
division                         Lightweight
method                                   SUB
round                                      1
time                                    1:24
referee                        John McCarthy
time_format                             12-3
reach                                    NaN
age                                      NaN
height                                  72.0
knockdowns                                 0
days_since_last_comp              216.108044
sub_attempts                               0
reversals                                  0
control                                    0
takedowns_landed                           0
takedowns_attempts                         0
sig_strike

In [12]:
lightweight["fighter"].nunique()

524

In [13]:
date_threshold = datetime.date(2021, 1, 1)
early_lightweight = lightweight[lightweight["date"] < date_threshold]
late_lightweight = lightweight[lightweight["date"] >= date_threshold]

In [14]:
early_lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
198,1997-02-07,0,rainy martinez,jerry bohlander,Orthodox,NaT,Lightweight,SUB,1,1:24,...,0,1.0,0,1,0.000000,0,84.0,0.000,1155.936792,21.246039
199,1997-02-07,1,jerry bohlander,rainy martinez,Orthodox,1974-02-12,Lightweight,SUB,1,1:24,...,0,4.0,2,0,0.750000,1,1858.0,0.000,1292.341803,29.965180
200,1997-02-07,0,wallid ismail,yoshiki takahashi,Orthodox,1968-02-23,Lightweight,U-DEC,2,3:00,...,0,1.0,0,1,0.000000,0,900.0,0.000,1147.756618,20.794729
201,1997-02-07,1,yoshiki takahashi,wallid ismail,Southpaw,1969-03-13,Lightweight,U-DEC,2,3:00,...,10,1.0,1,0,1.000000,0,900.0,0.000,1252.243382,29.205271
206,1997-02-07,0,nick sanzo,jerry bohlander,NaN,NaT,Lightweight,SUB,1,0:39,...,0,1.0,0,1,0.000000,0,39.0,0.000,1162.881928,21.710767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11743,2020-12-05,1,gabriel benitez,justin jaynes,Southpaw,1988-06-15,Lightweight,KO/TKO,1,4:06,...,5,10.0,1,0,0.600000,2,5342.0,0.600,1273.013267,29.423404
11754,2020-12-12,0,renato moicano,rafael fiziev,Orthodox,1989-05-21,Lightweight,KO/TKO,1,4:05,...,0,10.0,0,1,0.600000,3,4935.0,0.750,1240.856956,34.152111
11755,2020-12-12,1,rafael fiziev,renato moicano,Switch,1993-03-05,Lightweight,KO/TKO,1,4:05,...,1,4.0,3,0,0.750000,1,2131.0,1.000,1300.963413,33.694983
11764,2020-12-12,0,tony ferguson,charles oliveira,Orthodox,1984-02-12,Lightweight,U-DEC,3,5:00,...,3,18.0,0,2,0.833333,1,11985.0,0.625,1506.243446,38.479987


In [15]:
late_lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
11816,2021-01-20,0,mason jones,mike davis,Orthodox,1995-04-26,Lightweight,U-DEC,3,5:00,...,7,1.0,0,1,0.000000,0,900.0,-0.050000,1165.782229,20.474403
11817,2021-01-20,1,mike davis,mason jones,Orthodox,1992-10-07,Lightweight,U-DEC,3,5:00,...,4,3.0,2,0,0.666667,0,2340.0,1.000000,1246.438167,27.871592
11850,2021-01-23,0,matt frevola,arman tsarukyan,Orthodox,1990-06-11,Lightweight,U-DEC,3,5:00,...,0,5.0,0,1,0.400000,1,3660.0,0.500000,1186.406302,29.233458
11851,2021-01-23,1,arman tsarukyan,matt frevola,Orthodox,1996-10-11,Lightweight,U-DEC,3,5:00,...,9,4.0,3,0,0.750000,0,3600.0,0.750000,1343.102444,35.430054
11858,2021-01-23,0,dan hooker,michael chandler,Switch,1990-02-13,Lightweight,KO/TKO,1,2:30,...,0,16.0,0,2,0.625000,2,9945.0,0.375000,1338.823540,33.640663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2023-03-18,1,justin gaethje,rafael fiziev,Orthodox,1988-11-14,Lightweight,M-DEC,3,5:00,...,0,11.0,1,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578
14108,2023-04-15,0,lando vannata,daniel zellhuber,Orthodox,1992-03-14,Lightweight,U-DEC,3,5:00,...,2,13.0,0,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970
14109,2023-04-15,1,daniel zellhuber,lando vannata,Switch,1999-07-07,Lightweight,U-DEC,3,5:00,...,51,2.0,1,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804
14118,2023-04-15,0,clay guida,rafa garcia,Orthodox,1981-12-08,Lightweight,U-DEC,3,5:00,...,0,35.0,0,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986


Data Cleaning

In [16]:
cleaned_lightweight = lightweight

#null values
average_reach_lightweight = cleaned_lightweight[cleaned_lightweight['division'] == 'Lightweight']['reach'].mean()
cleaned_lightweight['reach'].fillna(average_reach_lightweight, inplace=True)

cleaned_lightweight.dropna(subset=['stance', 'age', 'height'], inplace=True)

#replace
def time_to_seconds(time_str):
    if pd.isnull(time_str):
        return np.nan
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

specific_value = 216.1080438291005
cleaned_lightweight.loc[cleaned_lightweight['days_since_last_comp'] == specific_value, 'days_since_last_comp'] = cleaned_lightweight['age']


cleaned_lightweight['time_seconds'] = cleaned_lightweight['time'].apply(time_to_seconds)

#remove columns
columns_to_ignore = ['date', 'fighter', 'opponent', 'dob', 'division', 'referee', 'time', 'time_format']
cleaned_lightweight.drop(columns=columns_to_ignore, inplace=True)


/tmp/ipykernel_26/1899271400.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lightweight['reach'].fillna(average_reach_lightweight, inplace=True)
/tmp/ipykernel_26/1899271400.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lightweight.dropna(subset=['stance', 'age', 'height'], inplace=True)
/tmp/ipykernel_26/1899271400.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [17]:
#label encoding 
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Label encode 'stance' column
if 'stance' in cleaned_lightweight.columns:
    cleaned_lightweight['stance_encoded'] = label_encoder.fit_transform(cleaned_lightweight['stance'])

# Label encode 'method' column
if 'method' in cleaned_lightweight.columns:
    cleaned_lightweight['method_encoded'] = label_encoder.fit_transform(cleaned_lightweight['method'])

# Drop the original 'stance' and 'method' columns
cleaned_lightweight.drop(columns=['stance', 'method'], inplace=True)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/tmp/ipykernel_26/2437616765.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [18]:
cleaned_lightweight

,result,round,reach,age,height,knockdowns,days_since_last_comp,sub_attempts,reversals,control,...,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill,time_seconds,stance_encoded,method_encoded
199,1,1,71.462766,8396.0,71.0,0,140.0,1,0,0,...,0,0.750000,1,1858.0,0.000000,1292.341803,29.965180,84,0,5
200,0,2,71.462766,10577.0,67.0,0,10577.0,0,0,0,...,1,0.000000,0,900.0,0.000000,1147.756618,20.794729,180,0,6
201,1,2,71.462766,10193.0,71.0,0,10193.0,0,0,0,...,0,1.000000,0,900.0,0.000000,1252.243382,29.205271,180,1,6
207,1,1,71.462766,8396.0,71.0,0,0.0,1,0,0,...,0,0.800000,1,1897.0,0.000000,1329.459875,31.301881,39,0,5
213,1,1,74.000000,8163.0,75.0,0,8163.0,0,0,0,...,0,1.000000,0,31.0,0.000000,1263.165486,29.205271,31,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,1,3,70.000000,12542.0,71.0,0,315.0,0,0,5,...,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578,300,0,3
14108,0,3,71.000000,11354.0,69.0,0,357.0,0,0,6,...,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970,300,0,6
14109,1,3,77.000000,8683.0,73.0,1,210.0,0,0,104,...,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804,300,2,6
14118,0,3,70.000000,15103.0,67.0,0,133.0,0,0,0,...,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986,300,0,6


In [19]:
cleaned_lightweight.dtypes

result                         int64
round                          int64
reach                        float64
age                          float64
height                       float64
knockdowns                     int64
days_since_last_comp         float64
sub_attempts                   int64
reversals                      int64
control                        int64
takedowns_landed               int64
takedowns_attempts             int64
sig_strikes_landed             int64
sig_strikes_attempts           int64
total_strikes_landed           int64
total_strikes_attempts         int64
head_strikes_landed            int64
head_strikes_attempts          int64
body_strikes_landed            int64
body_strikes_attempts          int64
leg_strikes_landed             int64
leg_strikes_attempts           int64
distance_strikes_landed        int64
distance_strikes_attempts      int64
clinch_strikes_landed          int64
clinch_strikes_attempts        int64
ground_strikes_landed          int64
g

# LinUCB 

### copy and edit/optimise after this markdown

In [20]:
lightweight.dtypes

result                         int64
round                          int64
reach                        float64
age                          float64
height                       float64
knockdowns                     int64
days_since_last_comp         float64
sub_attempts                   int64
reversals                      int64
control                        int64
takedowns_landed               int64
takedowns_attempts             int64
sig_strikes_landed             int64
sig_strikes_attempts           int64
total_strikes_landed           int64
total_strikes_attempts         int64
head_strikes_landed            int64
head_strikes_attempts          int64
body_strikes_landed            int64
body_strikes_attempts          int64
leg_strikes_landed             int64
leg_strikes_attempts           int64
distance_strikes_landed        int64
distance_strikes_attempts      int64
clinch_strikes_landed          int64
clinch_strikes_attempts        int64
ground_strikes_landed          int64
g

The new code

In [21]:
masterMLpublic.dropna(subset=["age", "height"], inplace=True)

In [22]:
# fix days_since_last_comp variable
specific_value = 216.1080438291005
masterMLpublic.loc[masterMLpublic["days_since_last_comp"] == specific_value, "days_since_last_comp"] = masterMLpublic["age"]

In [23]:
# removing draws because nobody wins
masterMLpublic = masterMLpublic[masterMLpublic["method"] != "DRAW"]
# removing dqs because they are not standard wins
masterMLpublic = masterMLpublic[masterMLpublic["method"] != "DQ"]

In [24]:
# Filtering out the lightweight division for analysis
lightweight = masterMLpublic[masterMLpublic["division"] == "Lightweight"]
lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
199,1997-02-07,1,jerry bohlander,rainy martinez,Orthodox,1974-02-12,Lightweight,SUB,1,1:24,...,0,4.0,2,0,0.750000,1,1858.0,0.000000,1292.341803,29.965180
200,1997-02-07,0,wallid ismail,yoshiki takahashi,Orthodox,1968-02-23,Lightweight,U-DEC,2,3:00,...,0,1.0,0,1,0.000000,0,900.0,0.000000,1147.756618,20.794729
201,1997-02-07,1,yoshiki takahashi,wallid ismail,Southpaw,1969-03-13,Lightweight,U-DEC,2,3:00,...,10,1.0,1,0,1.000000,0,900.0,0.000000,1252.243382,29.205271
207,1997-02-07,1,jerry bohlander,nick sanzo,Orthodox,1974-02-12,Lightweight,SUB,1,0:39,...,0,5.0,3,0,0.800000,1,1897.0,0.000000,1329.459875,31.301881
213,1997-05-30,1,tito ortiz,wes albritton,Orthodox,1975-01-23,Lightweight,KO/TKO,1,0:31,...,23,1.0,1,0,1.000000,0,31.0,0.000000,1263.165486,29.205271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2023-03-18,1,justin gaethje,rafael fiziev,Orthodox,1988-11-14,Lightweight,M-DEC,3,5:00,...,0,11.0,1,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578
14108,2023-04-15,0,lando vannata,daniel zellhuber,Orthodox,1992-03-14,Lightweight,U-DEC,3,5:00,...,2,13.0,0,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970
14109,2023-04-15,1,daniel zellhuber,lando vannata,Switch,1999-07-07,Lightweight,U-DEC,3,5:00,...,51,2.0,1,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804
14118,2023-04-15,0,clay guida,rafa garcia,Orthodox,1981-12-08,Lightweight,U-DEC,3,5:00,...,0,35.0,0,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986


In [25]:
average_reach_lightweight = lightweight["reach"].mean()
lightweight["reach"].fillna(average_reach_lightweight, inplace=True)

/tmp/ipykernel_26/2918575452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lightweight["reach"].fillna(average_reach_lightweight, inplace=True)


In [26]:
lightweight["fighter"].nunique()

493

In [27]:
import numpy as np
import pandas as pd

class ModifiedWinLossLinUCB:
    def __init__(self, num_arms, num_features, alpha=0.1):
        self.num_arms = num_arms  # This is actually the number of fights, not fighters
        self.num_features = num_features
        self.alpha = alpha
        self.A = [np.identity(num_features) for _ in range(self.num_arms)]  # One A matrix per fight
        self.b = [np.zeros((num_features, 1)) for _ in range(self.num_arms)]  # One b vector per fight

    def recommend(self, context):
        chosen_arm = None
        max_ucb = float('-inf')

        for arm in [0, 1]:  # Two possible arms: fighter 1 wins or fighter 2 wins
            x = context.reshape((self.num_features, 1))
            A_inv = np.linalg.inv(self.A[arm])
            theta = np.dot(A_inv, self.b[arm])
            ucb_value = x.T @ A_inv @ x

            # Calculate the UCB
            # Use item() to extract a scalar from a numpy array
            ucb = np.dot(theta.T, x) + self.alpha * np.sqrt(ucb_value.item())

            if ucb > max_ucb:
                max_ucb = ucb
                chosen_arm = arm

        return chosen_arm


    def update(self, fight_index, context, reward):
        x = context.reshape((self.num_features, 1))
        self.A[fight_index] += np.dot(x, x.T)
        self.b[fight_index] += reward * x


# Assume 'lightweight' is a pandas DataFrame containing your data
# Assume 'chosen_features' is defined as the list of features for the fighters
chosen_features = [
    "total_comp_time", "reach", "height", "age",
    "knockdowns", "sub_attempts", "reversals", "control",
    "takedowns_landed", "takedowns_attempts",
    "sig_strikes_landed", "sig_strikes_attempts",
    "total_strikes_landed", "total_strikes_attempts",
    "head_strikes_landed", "head_strikes_attempts",
    "body_strikes_landed", "body_strikes_attempts",
    "leg_strikes_landed", "leg_strikes_attempts",
    "distance_strikes_landed", "distance_strikes_attempts",
    "clinch_strikes_landed", "clinch_strikes_attempts",
    "ground_strikes_landed", "ground_strikes_attempts",
    "KO_losses", "days_since_last_comp", "lose_streak",
    "win_streak", "win_loss_ratio",
    "stamina", "num_fights", "trueskill", "elo"
]

# Ensure 'result' column exists and indicates the winner (1 for the winner, 0 for the loser)

num_fights = len(lightweight) // 2
num_features = len(chosen_features) * 2  # Features from both fighters
bandit = ModifiedWinLossLinUCB(num_fights, num_features)

total_reward = 0
correct_predictions = 0

for fight_index in range(num_fights):
    # Get the records for both fighters in the current fight
    fighter1_record = lightweight.iloc[fight_index * 2]
    fighter2_record = lightweight.iloc[fight_index * 2 + 1]

    # Convert features from both fighters to numeric types and handle non-numeric entries
    fighter1_features = pd.to_numeric(fighter1_record[chosen_features], errors='coerce').fillna(0)
    fighter2_features = pd.to_numeric(fighter2_record[chosen_features], errors='coerce').fillna(0)

    context = np.concatenate([fighter1_features, fighter2_features])
    # Predict the winner using the combined context
    chosen_arm = bandit.recommend(context)  # This will be 0 or 1

    # Determine the actual winner (arm 0 represents fighter 1 and arm 1 represents fighter 2)
    actual_winner = 0 if fighter1_record['result'] == 1 else 1

    # Update the bandit with the outcome of the fight
    reward = 1 if chosen_arm == actual_winner else 0
    bandit.update(fight_index // 2, context, reward)  # Update using fight index
    # bandit.update(fight_index, context, reward)

    # Track total reward and correct predictions
    total_reward += reward
    correct_predictions += (chosen_arm == actual_winner)


# Calculate accuracy
accuracy = correct_predictions / num_fights
print(f"Total reward: {total_reward}")
print(f"Accuracy: {accuracy}")


Total reward: 833
Accuracy: 0.6988255033557047


In [28]:
import numpy as np
import pandas as pd

class ModifiedContextualThompsonSampling:
    def __init__(self, num_arms, num_features, delta=0.5, R=0.01, epsilon=0.5, random_state=None):
        self.num_arms = num_arms
        self.num_features = num_features
        self.delta = delta
        self.R = R
        self.epsilon = epsilon
        self.random_state = random_state

        self.A = [np.identity(num_features) for _ in range(self.num_arms)]
        self.b = [np.zeros((num_features, 1)) for _ in range(self.num_arms)]

    def recommend(self, context):
        scores = np.zeros(self.num_arms)

        for arm in range(self.num_arms):
            A_inv = np.linalg.inv(self.A[arm])
            mu_hat = A_inv @ self.b[arm]
            v = self.R * np.sqrt(24 / self.epsilon * self.num_features * np.log(1 / self.delta))
            mu_tilde = np.random.multivariate_normal(mu_hat.flat, v**2 * A_inv)[..., np.newaxis]

            scores[arm] = context @ mu_tilde

        chosen_arm = np.argmax(scores)
        return chosen_arm

    def update(self, fight_index, context, reward):
        x = context.reshape((self.num_features, 1))
        self.A[fight_index] += np.dot(x, x.T)
        self.b[fight_index] += reward * x

# Assume 'lightweight' is a pandas DataFrame containing your data
# Assume 'chosen_features' is defined as the list of features for the fighters

# Ensure 'result' column exists and indicates the winner (1 for the winner, 0 for the loser)

num_fights = len(lightweight) // 2
num_features = len(chosen_features) * 2  # Features from both fighters
cts_bandit = ModifiedContextualThompsonSampling(num_fights, num_features)

total_reward = 0
correct_predictions = 0

for fight_index in range(num_fights):
    # Get the records for both fighters in the current fight
    fighter1_record = lightweight.iloc[fight_index * 2]
    fighter2_record = lightweight.iloc[fight_index * 2 + 1]

    # Convert features from both fighters to numeric types and handle non-numeric entries
    fighter1_features = pd.to_numeric(fighter1_record[chosen_features], errors='coerce').fillna(0)
    fighter2_features = pd.to_numeric(fighter2_record[chosen_features], errors='coerce').fillna(0)

    context = np.concatenate([fighter1_features, fighter2_features])
    # Predict the winner using the combined context
    chosen_arm = cts_bandit.recommend(context)  # This will be 0 or 1

    # Determine the actual winner (arm 0 represents fighter 1 and arm 1 represents fighter 2)
    actual_winner = 0 if fighter1_record['result'] == 1 else 1

    # Update the bandit with the outcome of the fight
    reward = 1 if chosen_arm == actual_winner else 0
    cts_bandit.update(fight_index, context, reward)  # Update using fight index

    # Track total reward and correct predictions
    total_reward += reward
    correct_predictions += (chosen_arm == actual_winner)

# Calculate accuracy
accuracy = correct_predictions / num_fights
print(f"Total reward: {total_reward}")
print(f"Accuracy: {accuracy}")


Total reward: 0
Accuracy: 0.0
